In [1]:
pip install -U sentence-transformers

     |████████████████████████████████| 51kB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 450kB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 4.6MB/s eta 0:00:01
     |████████████████████████████████| 686kB 9.1MB/s eta 0:00:01
     |████████████████████████████████| 870kB 9.4MB/s eta 0:00:01
  Stored in directory: /Users/sashaepelbaum/Library/Caches/pip/wheels/b4/ce/39/5bbda8ac34eb52df8c6531382ca077773fbfcbfb6386e5d66c
  Stored in directory: /Users/sashaepelbaum/Library/Caches/pip/wheels/1c/78/87/21be0303007ee5d1483df56703c9c7e5a44873e8f0c51d65f8
  Stored in directory: /Users/sashaepelbaum/Library/Caches/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers regex sacremoses
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import en_core_web_sm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import re

In [2]:
df = pd.read_csv('sentence_features.csv')

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [01:07<00:00, 6.01MB/s] 


In [4]:
X_text = df['text']
y_text = df['propaganda']

In [6]:
sentences_train, sentences_test, y_train, y_test = train_test_split(X_text, y_text, test_size=0.33, random_state=42)

In [25]:
sentences_train.shape

(10197,)

In [10]:
sentences_train = np.array(sentences_train)
sentences_test = np.array(sentences_test)

In [11]:
sentence_embeddings_train = model.encode(sentences_train)
sentence_embeddings_test = model.encode(sentences_test)

In [21]:
len(sentence_embeddings_train[0])

768

In [15]:
import sklearn
from sklearn.linear_model import LogisticRegression

In [30]:
# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression(C = 3, penalty = 'l2', solver='newton-cg', class_weight = 'balanced', max_iter = 1000)
# logreg_cv=GridSearchCV(logreg,grid,cv=10,scoring='f1_weighted')
logreg_embeddings_1 = logreg.fit(sentence_embeddings_train, y_train)
logreg_embeddings_preds_1 = logreg_embeddings_1.predict(sentence_embeddings_test)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, logreg_embeddings_preds_1))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, logreg_embeddings_preds_1, digits=3))

[[2609  959]
 [ 454 1001]]
                precision    recall  f1-score   support

non-propaganda      0.852     0.731     0.787      3568
    propaganda      0.511     0.688     0.586      1455

      accuracy                          0.719      5023
     macro avg      0.681     0.710     0.687      5023
  weighted avg      0.753     0.719     0.729      5023



In [59]:
# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg2=LogisticRegression(C = 5, penalty = 'l2', solver='newton-cg', class_weight = 'balanced', max_iter = 1000)
# logreg_cv=GridSearchCV(logreg,grid,cv=10,scoring='f1_weighted')
logreg_embeddings_2 = logreg2.fit(sentence_embeddings_train, y_train)
logreg_embeddings_preds_2 = logreg_embeddings_2.predict(sentence_embeddings_test)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, logreg_embeddings_preds_2))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, logreg_embeddings_preds_2, digits=3))

[[2612  956]
 [ 455 1000]]
                precision    recall  f1-score   support

non-propaganda      0.852     0.732     0.787      3568
    propaganda      0.511     0.687     0.586      1455

      accuracy                          0.719      5023
     macro avg      0.681     0.710     0.687      5023
  weighted avg      0.753     0.719     0.729      5023



In [32]:
from sklearn.linear_model import SGDClassifier

In [58]:
svd_sgd_clf = SGDClassifier(loss="log", penalty="l2", max_iter=200, class_weight = 'balanced')
svd_sgd_clf.fit(sentence_embeddings_train, y_train)
svd_sgd_predicts_embeddings = svd_sgd_clf.predict(sentence_embeddings_test)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, svd_sgd_predicts_embeddings))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, svd_sgd_predicts_embeddings, digits=3))

[[2101 1467]
 [ 318 1137]]
                precision    recall  f1-score   support

non-propaganda      0.869     0.589     0.702      3568
    propaganda      0.437     0.781     0.560      1455

      accuracy                          0.645      5023
     macro avg      0.653     0.685     0.631      5023
  weighted avg      0.743     0.645     0.661      5023

